In [1]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.tools as tls
import plotly.graph_objs as go
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import MinMaxScaler
from influxdb_client import InfluxDBClient
import optuna
import anvil.server
import anvil.media

In [2]:
anvil.server.connect("server_MI4VVJ5RM3X245Z3KNPIKZNY-5BPUVYDAD77UUJVI")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [3]:
@anvil.server.callable
def get_data(range_fetch_data):
    # Connection details
    url = 'http://192.168.7.90:8086'
    token = '7cdJZe84FUVh8bWKZ0uAV6VH1vcw5k4PM8Pvv_5EpsScmhU_A5ZaXSwvIp4vgWb6m222k6qghwTl8IOCZDVxFw=='
    org = 'UoA'
    start_time = '-' + range_fetch_data + 'm'
    # Connect to the InfluxDB instance
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = client.query_api()
    query ='from(bucket: "telegraf")\
    |> range(start: ' + start_time + ', stop: now())\
    |> filter(fn: (r) => r._measurement == "node_cpu_seconds_total")\
    |> filter(fn: (r) => r._field == "counter")\
    |> filter(fn: (r) => r.url == "http://192.168.8.133:9100/metrics")\
    |> filter(fn: (r) => r.mode == "softirq")\
    |> derivative(unit: 1s, nonNegative: false)\
    |> keep(columns: ["_time", "_measurement", "_value"])\
    |> rename(columns: {_value: "load1", _time: "Time"})\
    |> pivot(rowKey:["Time"], columnKey: ["_measurement"], valueColumn: "load1")'
    result = query_api.query_data_frame(org=org, query=query)
    result['Time'] = result['Time'].astype(str)
    result['Time'] = result['Time'].str.split('+').str[0] 
    result['Time'] = pd.to_datetime(result['Time'])
    result.index = result['Time']
    result = result.drop(columns = ['result', 'table', 'Time'])
    query_1 ='from(bucket: "telegraf")\
    |> range(start: ' + start_time + ', stop: now())\
    |> filter(fn: (r) => r._measurement == "node_network_receive_bytes_total")\
    |> filter(fn: (r) => r._field == "counter")\
    |> filter(fn: (r) => r.url == "http://192.168.8.133:9100/metrics")\
    |> filter(fn: (r) => r.device == "upfgtp")\
    |> derivative(unit: 1s, nonNegative: false)\
    |> keep(columns: ["_time", "_measurement", "_value"])\
    |> rename(columns: {_value: "load1", _time: "Time"})\
    |> pivot(rowKey:["Time"], columnKey: ["_measurement"], valueColumn: "load1")'
    result_1 = query_api.query_data_frame(org=org, query=query_1)
    result_1['Time'] = result_1['Time'].astype(str)
    result_1['Time'] = result_1['Time'].str.split('+').str[0] 
    result_1['Time'] = pd.to_datetime(result_1['Time'])
    result_1.index = result_1['Time']
    result_1 = result_1.drop(columns = ['result', 'table', 'Time'])
    result['node_network_receive_bytes_total'] = result_1['node_network_receive_bytes_total']
    result = result.to_json(orient='index')
    client.close()
    return result

In [8]:
@anvil.server.callable
def objective(trial, prediction_model, X_train, y_train, X_valid, y_valid, T, HORIZON):
    # Define the search space for hyperparameters
    units = trial.suggest_categorical('units', [50, 100, 200])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    epochs = trial.suggest_categorical('epochs', [100, 200])
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'RMSprop'])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    model = Sequential()
    if prediction_model == 'GRU':
        model.add(GRU(units = units, input_shape = (T, 1), activation = activation))
    elif prediction_model == 'LSTM':
        model.add(LSTM(units = units, input_shape = (T, 1), activation = activation))
    elif prediction_model == 'CNN':
        model.add(Conv1D(64, 1, activation=activation, input_shape=(T, 1)))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(units = units, activation=activation))
    elif prediction_model == 'MLP':
        model.add(Dense(units = units, activation=activation, input_dim=T))
    model.add(Dense(int(HORIZON)))
    model.compile(optimizer = optimizer, loss = 'mse', metrics=['accuracy'])
    model.summary() 
    model_fit = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_valid, y_valid), verbose = 0)
    # Evaluate the model accuracy on the validation set.
    score = model_fit.history['val_loss'][-1]
    return score

In [9]:
@anvil.server.callable
def model_optimized(prediction_model, units, activation, epochs, optimizer, batch_size, X_train, y_train, X_valid, y_valid, T, HORIZON):
    model = Sequential()
    if prediction_model == 'GRU':
        model.add(GRU(units = units, input_shape = (T, 1), activation = activation))
    elif prediction_model == 'LSTM':
        model.add(LSTM(units = units, input_shape = (T, 1), activation = activation))
    elif prediction_model == 'CNN':
        model.add(Conv1D(64, 1, activation=activation, input_shape=(T, 1)))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(units = units, activation=activation))
    elif prediction_model == 'MLP':
        model.add(Dense(units = units, activation=activation, input_dim=T))
    model.add(Dense(int(HORIZON)))
    model.compile(optimizer = optimizer, loss = 'mse', metrics=['accuracy'])
    model.summary() 
    model_fit = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_valid, y_valid), verbose = 0)
    return model, model_fit

In [10]:
@anvil.server.callable
def time_series_prediction_with_data_new(prediction_model, column, json_data):
    df = pd.read_json(json_data, orient='index')    
    if column == 'node_cpu_seconds_total':
        df = df.drop(columns = ['node_network_receive_bytes_total'])
        data_set_selection = 'node_cpu_seconds_total'
    elif column == 'node_network_receive_bytes_total':
        df = df.drop(columns = ['node_cpu_seconds_total'])
        data_set_selection = 'node_network_receive_bytes_total'
    length_df = len(df)
    train_rows = int(0.7*length_df)
    valid_rows = int(0.1*length_df)
    test_rows = length_df - train_rows - valid_rows
    train = df.copy()[:train_rows]
    train_rows_new = train_rows + 1
    valid = df.copy()[train_rows:train_rows + valid_rows]
    test = df.copy()[train_rows + valid_rows:]
    scaler = MinMaxScaler()
    train[data_set_selection] = scaler.fit_transform(train)
    valid[data_set_selection] = scaler.fit_transform(valid)
    test[data_set_selection] = scaler.fit_transform(test)
    T = 2 # Let the input be a vector of the previous 12 seconds of the values.
    HORIZON = 1 # one-step-ahead prediction
    train_shifted = train.copy()
    train_shifted['y_t+1'] = train_shifted[data_set_selection].shift(-1, freq = '6s')
    for t in range(1, T+1):
        train_shifted[str(T-t)] = train_shifted[data_set_selection].shift(T-t, freq = '6s')
    y_col = 'y_t+1'
    X_cols = []
    for i in range(-T+1, 1):
        if i != 0:
            X_cols.append('t{}'.format(i))
        else:
            X_cols.append('t')
    train_shifted.columns = ['original'] + [y_col] + X_cols
    train_shifted = train_shifted.dropna(how = 'any')
    y_train = np.array(train_shifted[y_col]) 
    X_train = np.array(train_shifted[X_cols])
    X_train = X_train.reshape(X_train.shape[0], T, 1)
    valid_shifted = valid.copy()
    test_shifted = test.copy()
    valid_shifted['y_t+1'] = valid_shifted[data_set_selection].shift(-1, freq = '6s')
    test_shifted['y_t+1'] = test_shifted[data_set_selection].shift(-1, freq = '6s')
    for t in range(1, T+1):
        valid_shifted[str(T-t)] = valid_shifted[data_set_selection].shift(T-t, freq = '6s')
    for t in range(1, T+1):
        test_shifted[str(T-t)] = test_shifted[data_set_selection].shift(T-t, freq = '6s')
    valid_shifted.columns = ['original'] + [y_col] + X_cols
    test_shifted.columns = ['original'] + [y_col] + X_cols
    valid_shifted = valid_shifted.dropna(how = 'any')
    test_shifted = test_shifted.dropna(how = 'any')
    y_valid = np.array(valid_shifted[y_col])
    y_test = np.array(test_shifted[y_col])
    X_valid = np.array(valid_shifted[X_cols])
    X_test = np.array(test_shifted[X_cols])
    X_valid = X_valid.reshape(X_valid.shape[0], T, 1)
    X_test = X_test.reshape(X_test.shape[0], T, 1)
    func = lambda trial: objective(trial, prediction_model, X_train, y_train, X_valid, y_valid, T, HORIZON)
    study = optuna.create_study(direction="minimize")
    study.optimize(func, n_trials=108, timeout=None)
    trial = study.best_trial
    units = trial.params['units']
    activation = trial.params['activation']
    epochs = trial.params['epochs']
    optimizer = trial.params['optimizer']
    batch_size = trial.params['batch_size']
    model, model_fit = model_optimized(prediction_model, units, activation, epochs, optimizer, batch_size, X_train, y_train, X_valid, y_valid, T, HORIZON)
    predictions = model.predict(X_test)
    #valid_loss
    val_loss = model_fit.history['val_loss'][-1]
    val_loss_ui= 'Validation evaluation--> ' + 'Validation_Loss: ' + str(val_loss*100) + '%'
    best_parameters = 'Optimized Parameters--> ' + 'Units: ' + str(units) + ',   Activation Function: ' + str(activation) + ',   Epochs: ' + str(epochs) + ',   Optimizer: ' + str(optimizer) + ',   Batch Size: ' + str(batch_size)
    test_predicted_df = pd.DataFrame(predictions, columns = ['t+' + str(t) for t in range(1, int(HORIZON) + 1)])
    test_predicted_df['Time'] = test_shifted.index
    test_predicted_df = pd.melt(test_predicted_df, id_vars = 'Time', value_name = 'predicted', var_name = 'h')
    test_predicted_df['test'] = np.transpose(y_test).ravel()
    test_predicted_df[['predicted', 'test']] = scaler.inverse_transform(test_predicted_df[['predicted', 'test']])
    train_df = pd.DataFrame()
    train_df['Time'] = train_shifted.index
    train_df['train'] = np.transpose(y_train).ravel()
    train_df[['train']] = scaler.inverse_transform(train_df[['train']])
    valid_df = pd.DataFrame()
    valid_df['Time'] = valid_shifted.index
    valid_df['valid'] = np.transpose(y_valid).ravel()
    valid_df[['valid']] = scaler.inverse_transform(valid_df[['valid']])
    fig, ax = plt.subplots()
    ax.plot(test_predicted_df['Time'], test_predicted_df['predicted'],'--r', label = 'predicted')
    ax.plot(test_predicted_df['Time'], test_predicted_df['test'],'-b', label = 'test')
    ax.plot(train_df['Time'], train_df['train'],'-y', label = 'train')
    ax.plot(valid_df['Time'], valid_df['valid'],'-y', label = 'train')
    plt.legend()
    plt.ylabel(data_set_selection)
    plt.xlabel('Time')
    plotly_fig = tls.mpl_to_plotly(fig)
    data = plotly_fig['data'][0]
    data1 = plotly_fig['data'][1]
    data2 = plotly_fig['data'][2]
    data3 = plotly_fig['data'][3]
    plot_predicted = go.Scatter(x=data['x'], y=data['y'], mode='lines', name = 'predicted')
    plot_test = go.Scatter(x=data1['x'], y= data1['y'], mode='lines', name = 'test')
    plot_train = go.Scatter(x=data2['x'], y= data2['y'], mode='lines', name = 'train')
    plot_valid = go.Scatter(x=data3['x'], y= data3['y'], mode='lines', name = 'valid')
    return plot_predicted, plot_test, plot_train, plot_valid, val_loss_ui, best_parameters

[I 2023-07-13 02:47:25,298] A new study created in memory with name: no-name-795f3ba3-01aa-41aa-92af-03255daf06af


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 200)               121800    
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 122,001
Trainable params: 122,001
Non-trainable params: 0
_________________________________________________________________


[I 2023-07-13 02:47:59,806] Trial 0 finished with value: 0.01156475581228733 and parameters: {'units': 200, 'activation': 'tanh', 'epochs': 200, 'optimizer': 'sgd', 'batch_size': 128}. Best is trial 0 with value: 0.01156475581228733.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 100)               30900     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 31,001
Trainable params: 31,001
Non-trainable params: 0
_________________________________________________________________


[I 2023-07-13 02:48:26,964] Trial 1 finished with value: 0.009596362709999084 and parameters: {'units': 100, 'activation': 'tanh', 'epochs': 100, 'optimizer': 'sgd', 'batch_size': 32}. Best is trial 1 with value: 0.009596362709999084.


...

[I 2023-07-13 02:52:30,858] Trial 0 finished with value: 0.01082661934196949 and parameters: {'units': 50, 'activation': 'relu', 'epochs': 200, 'optimizer': 'adam', 'batch_size': 128}. Best is trial 0 with value: 0.01082661934196949.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 2, 64)             128       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1, 64)            0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 200)               13000     
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 13,329
Trainable params: 13,329
Non-trainable params: 0
__________________________________________________

[I 2023-07-13 02:52:46,805] Trial 1 finished with value: 0.010789381340146065 and parameters: {'units': 200, 'activation': 'relu', 'epochs': 100, 'optimizer': 'adam', 'batch_size': 32}. Best is trial 1 with value: 0.010789381340146065.


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 2, 64)             128       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 1, 64)            0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 200)               13000     
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 13,329
Trainable params: 13,329
Non-trainable params: 0
__________________________________________________

C:\ProgramData\Anaconda3\lib\site-packages\plotly\matplotlylib\renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.

